<img src="../../kerchunk.png" width=500 alt="Kerchunk Logo"></img>

# Kerchunk Basics

## Overview

This notebook is intended as an introduction to using kerchunk.
In this tutorial we will:
- Scan a single NetCDF file to create a kerchunk virtual dataset
- Learn how to use the output  using `xarray` and `fsspec`.

While this notebook only examines using kerchunk on a single NetCDF file, kerchunk can be used to create virutal zarr datasets from collections of many input files. In the following notebook, we will demonstrate this. 

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Introduction to Xarray](https://foundations.projectpythia.org/core/cartopy/cartopy.html) | Helpful | Basic features |

- **Time to learn**: 60 minutes
---

## Imports


In [ ]:
import fsspec
import kerchunk.hdf
import ujson
import xarray as xr

### Parse a single NetCDF file with kerchunk

Below we will access a NetCDF file stored on the AWS cloud. This dataset is a single time slice of a climate downscaled product for Alaska.

In [ ]:
# Options passed to fsspec
so = dict(anon=True, default_fill_cache=False, default_cache_type="first")


# Input URL to dataset. Note this is a netcdf file stored on s3 (cloud dataset).
url = "s3://wrf-se-ak-ar5/ccsm/rcp85/daily/2060/WRFDS_2060-01-01.nc"

# Uses kerchunk to scan through the netcdf file to create kerchunk mapping and then save output as .json
# Note: In this example, we write the kerchunk output to a .json file.
# You could also keep this information in memory and pass it to fsspec
with fsspec.open(url, **so) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, url, inline_threshold=100)
    h5chunks.translate()
    with open("single_file_kerchunk.json", "wb") as f:
        f.write(ujson.dumps(h5chunks.translate()).encode())

#### Load Kerchunk Reference File

In [ ]:
# use fsspec to create filesystem from .json reference file
fs = fsspec.filesystem(
    "reference",
    fo="./single_file_kerchunk.json",
    remote_protocol="s3",
    remote_options=dict(anon=True),
    skip_instance_cache=True,
)

# load kerchunked dataset with xarray
ds = xr.open_dataset(fs.get_mapper(""), engine="zarr")

### Plot Dataset

In [ ]:
ds.TMAX.plot()